In this notebook we are going to explore the chains inside the LangChain framework. We are going to explore the following elements:

TODO: Dame euna explciación en inglés para cada uno de estos elementos. Sé breve.
-   **LLMChain**: A single chain that sends input to a language model using a prompt template and returns the model’s output. It is the basic building block for interacting with LLMs.
-   **SequentialChains**: Chains that execute multiple sub-chains in sequence, passing the output of one as the input to the next. Useful for building multi-step workflows.
    -   **SimpleSequentialChain**: A simplified version of a sequential chain that automatically passes each chain’s output to the next. Best for linear, straightforward pipelines with minimal configuration.
    -   **SequentialChain**: A more flexible sequential chain that allows explicit mapping of input and output variables between steps, supporting complex workflows with multiple inputs and outputs.
-   **RouterChain**: A chain that dynamically decides which sub-chain to run based on the input. It acts as a “router” to direct tasks to the appropriate specialized chain or model.

# LLMChain

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

chat_model = ChatOllama(model="qwen2.5:3b", temperature=0.0)
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain = LLMChain(llm=chat_model, prompt=prompt)
product = "Queen Size Sheet Set"
response = chain.run(product)
print(response)

A suitable and descriptive name for a company that specializes in making Queen Size Sheet Sets could be "Queenly Sheets." This name not only reflects the product category but also conveys elegance, comfort, and quality. Another option could be "Royal Rest," which combines the idea of luxury with rest and relaxation.


# SimpleSequentialChain

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

llm = ChatOllama(model="qwen2.5:3b", temperature=0.0)
first_prompt = ChatPromptTemplate.from_template("What is a good name for a company that makes {product}?")
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True)
product = "Queen Size Sheet Set"
response = overall_simple_chain.run(product)
print(response)



> Entering new SimpleSequentialChain chain...
A creative and catchy name for a company that specializes in making Queen Size Sheet Sets could be "Queenly Sheets." This name not only reflects the product category but also conveys elegance, comfort, and quality. Another option could be "Royal Rest," which combines the idea of royalty with rest and relaxation. Both names aim to capture the essence of luxury associated with queen size bedding sets.
"Queenly Sheets" or "Royal Rest": catchy names for a company specializing in luxurious Queen Size Sheet Sets, evoking elegance, comfort, and quality.

> Finished chain.
"Queenly Sheets" or "Royal Rest": catchy names for a company specializing in luxurious Queen Size Sheet Sets, evoking elegance, comfort, and quality.


# SequentialChain

In [7]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

chat_model = ChatOllama(model="qwen2.5:3b", temperature=0.0)
# Prompt template 1: Translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=chat_model, prompt=first_prompt, output_key="English_Review")
# Prompt template 2: summarize review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=chat_model, prompt=second_prompt, output_key="summary")
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# Prompt template 3: detect language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=chat_model, prompt=third_prompt,
                       output_key="language"
                      )
# Prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=chat_model, prompt=fourth_prompt,output_key="followup_message")


# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

review = """Me encantó este producto! La calidad es excelente y superó mis expectativas. Lo recomiendo a todos."""
response = overall_chain(review)
print(response)

C:\Users\aleja\AppData\Local\Temp\ipykernel_30908\1378626460.py:52: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.
{'Review': 'Me encantó este producto! La calidad es excelente y superó mis expectativas. Lo recomiendo a todos.', 'English_Review': 'I loved this product! The quality is excellent and exceeded my expectations. I recommend it to everyone.', 'summary': "The reviewer highly praises the product's excellent quality that surpassed their expectations and recommends it to others.", 'followup_message': 'Resumen en español: El revisor elogia la excelente calidad del producto que superó sus expectativas y recomienda su compra a otros.\n\nResponse follow-up (in Spanish): La experiencia de este cliente es un testimonio claro de la excelencia del producto. Su recomendación nos anima a considerarlo como una opción segura para aquellos en búsqueda de alta calidad.'}


We can observe that we got a dict as output, where each key is each output of a single chain.

# RouterChain

In [ ]:
# Define a prompt for each subject area

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

# Create a list of prompt infos

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

chat_model = ChatOllama(model="qwen2.5:3b", temperature=0.0)
destination_chains = {}
# Create a dict of destination chains. Each key is the name of the subject area
# and each value is an LLMChain with the corresponding prompt template
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(
        llm=chat_model,
        prompt=prompt_template
    )
    destination_chains[name] = chain

# Create a string that lists the possible destinations
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [11]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [ ]:
# Define the router prompt template
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
from langchain.prompts import PromptTemplate
# Create the router prompt
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# Create the router chain and the overall multi-prompt chain
router_chain = LLMRouterChain.from_llm(chat_model, router_prompt)
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\aleja\AppData\Local\Temp\ipykernel_30908\3199927026.py:15: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [13]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': "What is black body radiation and how does it relate to Planck's law?"}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an object due solely to its temperature, without any specific absorption or emission characteristics. In other words, if an object absorbs all incident electromagnetic radiation regardless of frequency or polarization, then it is a perfect black body.\n\nPlanck's law describes the distribution of energy radiated by a black body in thermal equilibrium at a given temperature. The key feature of Planck's law is that it shows how the intensity of emitted radiation depends on both wavelength and temperature. Specifically, the law states that the spectral radiance (the amount of power per unit area per unit solid angle per unit frequency) of a black body increases with increasing frequency up to a peak value, after which it decreases.\n\nPlanck's law is given by:\n\n\\[ B(\\lambda, T) = \\frac{2hc^2}{\\lambda^5} \\cdot \\frac{1}{e^{h\\nu / kT} - 1} \\]\n\nwhere:\n- \\(B(\\lambda, T)\\) is the spectral radiance,\n- \\(h\